# Werner State monogamy #

WORK IN PROGRESS

In [ ]:
n = 3
d = [2,2,2]
marg1 = (rho_AB, [False, False, True])
marg2 = (rho_BC, [True, False, False])
(state, rho_g) = mg.construct_global_state(n,d,[marg1,marg2])

In [ ]:
if state == OPTIMAL:
    print(rho_g)
else:
    print("There was no solution found, solver returned: ", state)